In [9]:
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import pandas as pd


In [2]:
import neuralxpresso2 as nx
YT_LINK = "https://www.youtube.com/watch?v=m70UInZKJjU"
cb_palette = ['#CA3435', '#ff7f00', '#9370DB', '#40E0D0', '#1f77b4', '#9467bd', '#7f7f7f']


npx = nx.NeuralXpressoSession(yt_link=YT_LINK)
result = npx.run_analysis()

2023-03-27 15:46:05.034548: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-27 15:46:05.213621: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 14ms/step


In [7]:
df_character = result['character_overview']
df_character

,person_ID,appearances,Angry,Disgust,Fear,Happy,Sad,Surprise,Neutral
0,1,49,0.139440,0.022268,0.106683,0.250476,0.070885,0.226807,0.183441
1,2,48,0.066614,0.021204,0.137152,0.433164,0.044026,0.025478,0.272362
2,3,1,0.144567,0.016766,0.222988,0.184438,0.278748,0.013806,0.138688
3,4,7,0.049333,0.007716,0.033447,0.685559,0.069034,0.037845,0.117066
4,5,1,0.103874,0.007410,0.176604,0.432577,0.039263,0.018609,0.221663


In [64]:
df_video = result['video_overview']
df_video

,frame,person_ID,emotion,probability
0,0,1,Angry,0.048646
1,0,2,Angry,0.184672
2,1,2,Angry,0.022843
3,1,1,Angry,0.287634
4,2,1,Angry,0.131788
...,...,...,...,...
737,53,2,Neutral,0.271489
738,54,4,Neutral,0.043647
739,54,2,Neutral,0.265225
740,55,2,Neutral,0.131432


In [96]:
def get_df_single_person(df_video, ID):
    df_single_person = df_video[df_video['person_ID'] == ID]
    df_single_person['moving_avg'] = df_single_person['probability'].rolling(window=10, center=True).mean()
    return df_single_person



In [179]:

def get_emotion_landscape(df_single_person):
    # Create the plot 
    fig = px.area(df_single_person, x="frame", y="moving_avg", color="emotion",
                color_discrete_sequence=cb_palette, hover_data={"text": df_single_person['emotion']})

    # Update the layout 
    fig.update_layout(
        title={
            'text': '<b>Probability of emotion per frame</b>',
            'font': {'size': 24, 'color': 'black'},
            'x': 0.5, #center header
            'y': 0.95 #center header
        },
        xaxis_title='Frames',
        yaxis_title='Probability',
        legend_title='Emotion',
        font=dict(family='Arial', size=14),
        margin=dict(l=50, r=50, t=100, b=50),
        plot_bgcolor='white',
        xaxis=dict(dtick=1)  #set the x-axis step 
    )

    # Update the legend with customizations
    fig.update_traces(
        hovertemplate='<br>'.join([
            'Emotion: %{fullData.name}',
            'Frame: %{x:.2f} ',
            'Probability: %{y:.2f}'
        ]),
        hoverlabel=dict(bgcolor='white', font_size=14),
        line=dict(width=2),
        showlegend=True,
        stackgroup='one',
        hoverinfo='all'
    )

    return fig


In [180]:
df_single_person = get_df_single_person(df_video, ID=1)

/var/folders/f3/dnfpyp3918b33k59tvpmpcl40000gn/T/ipykernel_79056/1087280200.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [181]:
# Define a function that returns the emotion with the highest probability for a given row
def get_max_emotion(row):
    return row['emotion'][np.argmax(row['probability'])]

def get_df_radar(df_single_person):
    # Apply the function to each row of the DataFrame
    max_prob_rows = df_single_person.groupby('frame')['probability'].idxmax().reset_index()
    max_prob_df = df_single_person.loc[max_prob_rows['probability']]
    emotions_counts = max_prob_df['emotion'].value_counts().reset_index()
    return df_radar


In [183]:
# Define the color palette
cb_palette = ['#40E0D0', '#ff7f00', '#9370DB', '#CA3435', '#1f77b4', '#9467bd', '#7f7f7f']

def get_radar_plot(df_radar):
    # create the plot
    fig = go.Figure(data=go.Scatterpolar(
        r=df_radar['emotion'].tolist(),
        theta=df_radar['index'].tolist(),
        fill='toself',
        name='',
        line_color=cb_palette[0],
        fillcolor=cb_palette[0],
        hovertemplate='%{theta}: %{r}'
    ))

    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, max(df_radar['emotion'])],
                showticklabels=False,
                showgrid=False,
                ticks=''
            ),
            angularaxis=dict(
                tickfont=dict(size=14),
                rotation=90,
                direction='clockwise'
            )
        ),
        showlegend=False,
        height=400,
        margin=dict(t=80, b=50, l=50, r=50),
        font=dict(size=16)
    )

    fig.update_layout(
        title={
            'text': "<span style='font-size: 24px'>Prevalence Of Feelings During The Video</span>",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        paper_bgcolor='white',
        plot_bgcolor='white',
    )

    return fig



In [186]:
df_single_person = get_df_single_person(df_video, ID=1)
df_radar = get_df_radar(df_single_person)

# Create two traces and an image
image_array = result['portraits'][1]
fig_area = get_emotion_landscape(df_single_person)
#fig_radar = get_df_radar(df_radar)


# Create a 3x1 grid of subplots with custom column widths
fig = make_subplots(rows=1, cols=3, column_widths=[0.2, 0.6, 0.2])

# Add trace1, trace2, and the image to their respective subplots
fig.add_trace(px.imshow(image_array).data[0], row=1, col=1)
for i in range(7):
    fig.add_trace(fig_area.data[i], row=1, col=2)
#fig.add_trace(fig_radar, row=1, col=3)

fig.add_trace

# Update layout
fig.update_layout(title='Two Graphs and an Image Side by Side')

# Show the figure
fig.show()

/var/folders/f3/dnfpyp3918b33k59tvpmpcl40000gn/T/ipykernel_79056/1087280200.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

